In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import utils as uti

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
### Make a list of files

In [ ]:
%%time
exp='c64_gwR2_ne30pg3_FMTHIST_topfix_rdgres_x01'

A = uti.MakeDict4Exp( exp=exp  , user='juliob', subd='hist' , 
                     hsPat='cam.h2aZonal' , ymdPat='*' ,verbose=True, open_dataset=True )
#print(B.X.time.values)
zAX = A.X

In [ ]:
%%time
############
so4_a1=zAX.so4_a1.values
so4_a2=zAX.so4_a2.values
so4_a3=zAX.so4_a3.values

dms=zAX.DMS.values
n2o=zAX.N2O.values
co2mr =zAX.CO2.values
ch4=zAX.CH4.values
cfc12=zAX.CFC12.values

h2so4=zAX.H2SO4.values



In [ ]:
#ps_x =Dx.PS.values
#lon_a=A.X.lon.values
lat_a=A.X.lat.values
lev_a=A.X.lev.values
zlev_a=-7.0*np.log( lev_a/1_000. )


In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

AA = so4_a1
ThisFld='so4_a1'
FigName = f"{ThisFld}_aeroGHG.png"
print( FigName) 

# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=4,4
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(AA[0,:,:])
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(25, 19))


ulev=1.0e-9 * np.linspace( 0,2,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=12

plotTop=25

tees=[0,2,5,10,20,50,100,200,300,400,500,600,700]

titles = []
for i in np.arange( len(tees) ):
    titles.append( f'{ThisFld} day={str(tees[i])}')

print( titles )


n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

Pu.save_link(png_filename = FigName )


In [ ]:
#####


In [ ]:
#####


In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))
AA = so4_a2
ThisFld='so4_a2'
FigName = f"{ThisFld}_aeroGHG.png"
print( FigName) 



# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=4,4
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(AA[0,:,:])
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 19))


ulev=1.0e-10 * np.linspace( 0,2,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=12

plotTop=25

tees=[0,2,5,10,20,50,100,200,300,400,500,600,700]

titles = []
for i in np.arange( len(tees) ):
    titles.append( f'{ThisFld} day={str(tees[i])}')

print( titles )


n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

Pu.save_link(png_filename = FigName )


In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))
AA = so4_a3
ThisFld='so4_a3'
FigName = f"{ThisFld}_aeroGHG.png"
print( FigName) 



# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=4,4
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(AA[0,:,:])
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 19))


ulev=1.0e-11 * np.linspace( 0,5,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=12

plotTop=25

tees=[0,2,5,10,20,50,100,200,300,400,500,600,700]

titles = []
for i in np.arange( len(tees) ):
    titles.append( f'{ThisFld} day={str(tees[i])}')

print( titles )


n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

Pu.save_link(png_filename = FigName )


In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

AA = n2o
ThisFld='n2o'
FigName = f"{ThisFld}_aeroGHG.png"
print( FigName) 

# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=4,4
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(AA[0,:,:])
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 19))


ulev=3.2e-7 * np.linspace( 0,1,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=12

plotTop=25

tees=[0,2,5,10,20,50,100,200,300,400,500,600,700]

titles = []
for i in np.arange( len(tees) ):
    titles.append( f'{ThisFld} day={str(tees[i])}')

print( titles )


n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

Pu.save_link(png_filename = FigName )

In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

AA = cfc12 
ThisFld='cfc12'
FigName = f"{ThisFld}_aeroGHG.png"
print( FigName) 

# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=4,4
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(AA[0,:,:])
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 19))


ulev=1.e-10 *  np.linspace( 0,4,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=12

plotTop=25

tees=[0,2,5,10,20,50,100,200,300,400,500,600,700]

titles = []
for i in np.arange( len(tees) ):
    titles.append( f'{ThisFld} day={str(tees[i])}')

print( titles )


n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

Pu.save_link(png_filename = FigName )

In [ ]:
%%time
exp='c64_gwR2_ne30pg3_FMTHIST_topfix_rdgres_x01'

B = uti.MakeDict4Exp( exp=exp  , user='juliob', subd='hist' , 
                     hsPat='cam.h2aZonal' , ymdPat='*' ,verbose=True, open_dataset=True )
#print(B.X.time.values)
so4_a1=B.X.so4_a1.values

In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

AA = CO2mr * 1e6 # ppm
ThisFld='co2'
FigName = f"{ThisFld}_aeroGHG.png"
print( FigName) 

# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=4,4
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(AA[0,:,:])
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 19))


ulev=np.linspace( 340,360,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=12

plotTop=25

tees=[0,2,5,10,20,50,100,200,300,400,500,600,700]

titles = []
for i in np.arange( len(tees) ):
    titles.append( f'CO2 day={str(tees[i])}')

print( titles )


n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a, AA[tees[n-1],:,:],levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a, AA[tees[n-1],:,:] ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

Pu.save_link(png_filename = FigName )

In [ ]:
n2o.shape

In [ ]:

print( B.X.time[31].values) 
print( A.X.time[31].values) 


In [ ]:
%%time
so4_a1_B=B.X.so4_a1.values

In [ ]:

plt.plot( so4_a1_B[:,20,100] )
plt.plot( so4_a1[:,20,100] )
plt.plot( so4_a1[:-1,20,100]- so4_a1_B[:,20,100])


In [ ]:
for y in [1985,1986]:
    for m in np.arange(start=1,stop=13):
        yyyy=str(y)
        mm=str(m).zfill(2)
        ymdPat =f'{yyyy}-{mm}-*'
        f=f'/glade/derecho/scratch/juliob/archive/c64_gwR2_ne30pg3_FMTHIST_topfix_rdgres_x01/atm/hist/c64_gwR2_ne30pg3_FMTHIST_topfix_rdgres_x01.cam.h2aZonal.{yyyy}-{mm}.nc'
        print(f)


In [ ]:
plt.contourf(so4_a1[:,:,100].T)
plt.ylim(92,85)
plt.xlim(0,60)
plt.colorbar()

In [ ]:
plt.plot( so4_a1[:,88,100] )

In [ ]:
#plt.plot( Atopo.MXDIS[0,:] )
plt.plot( Btopo.ANGLX.values.flatten() - Atopo.ANGLX.values.flatten() )


In [ ]:

exp_Val='ERA5'

In [ ]:
########################
# Some massaging
########################


print( f'{A.exp} Years {len(A.X.time) /12. }')

#ps_x =Dx.PS.values
lon_a=A.X.lon.values
lat_a=A.X.lat.values
lev_a=A.X.lev.values
zlev_a=-7.0*np.log( lev_a/1_000. )



In [ ]:
season='djf'
UUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='U',return_time=True)
TTa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='T',return_time=True)



In [ ]:
UUa_sz=np.average( UUa_s , axis=2 )
TTa_sz=np.average( TTa_s , axis=2 )


In [ ]:
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False

Uval = Val.data(fld='U',season=season,mgrid=mgrid,zlev=True ) #, Years='1996' ) #, Years='2000' )
UUv_s , zlev_v,lat_v,lon_v = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon']
UUv_sz=np.average( UUv_s , axis=2 )

Tval = Val.data(fld='T',season=season,mgrid=mgrid,zlev=True ) #, Years='1996' ) #, Years='2000' )
TTv_s , zlev_v,lat_v,lon_v = Tval['aa'], Tval['lev'], Tval['lat'], Tval['lon']
TTv_sz=np.average( TTv_s , axis=2 )



In [ ]:
print(f'STOP ')

In [ ]:
print(f'STOP ')

In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))


# Create 2D arrays
Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
nxplo,nyplo=2,2
# Adding the hatching for the sponge layer
sponge_layer = np.zeros_like(UUa_sz)
sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1


fig = plt.figure(figsize=(15, 12))


ulev=np.linspace( -60,140,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=18

"""
titles = [ f"Control: $\Delta x$=100km (ne30) {season.upper()}" ,
          f"$\Delta x$=28km (ne120) {season.upper()} " ,
          f"$\Delta x$=14km (ne240) {season.upper()} " ,
          f"Validation <{exp_Val}> {season.upper()}  " ]
"""
titles = [ f"Control <{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" ,
          f"Control <{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}" ,
          f"Validation <{exp_Val}> {season.upper()}  " ,
          f"Validation <{exp_Val}> {season.upper()}  " ]
plotTop=82

n=1
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,UUa_sz ,levels=ulev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,UUa_sz ,levels=ulev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)


n=3
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=ulev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=ulev, colors='black')
#ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_title( titles[n-1] ,fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

n=2
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,TTa_sz ,levels=tlev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,TTa_sz ,levels=tlev, colors='black')
#ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_title( titles[n-1],fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')

# Sponge layer hatching
ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)


n=4
Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, TTv_sz ,levels=tlev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, TTv_sz ,levels=tlev, colors='black')
#ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_title( titles[n-1] ,fontsize=titlesize )
ax1.set_ylim(0,plotTop)
cb=plt.colorbar(co1)
ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
    fontsize=20, fontweight='bold', va='top')
